In [ ]:
from pynq import Overlay
from pynq import DefaultIP

In [ ]:
class MagicSeqdDriver (DefaultIP):

    def __init__(self, description):
        super().__init__(description=description)

        ### Bit Layout start bit
        self.BL_COL_ST  =  2 
        self.BL_ROW_ST  =  6 
        self.BL_BNK_ST = 16 
        ### Bit Layout size
        self.BIT_COL_SZ = 4
        self.BIT_ROW_SZ = 8
        self.BIT_BNK_SZ = 2

        self.REG_CTRL     = (0,0,0)
        self.REG_ST       = (0,1,0)
        self.REG_MAINCNT  = (0,2,0)
        self.REG_ENDCNT   = (0,3,0)
        self.REG_DMA_ADDR = (0,4,0)
        self.REG_DFX_ADDR = (0,5,0)

        #### the row must be change to match the slot
        self.SLOT_SRC_ADDR = (1,0,0)
        self.SLOT_SRC_SIZE = (1,0,1)
        self.SLOT_DES_ADDR = (1,0,2)
        self.SLOT_DES_SIZE = (1,0,3)
        self.SLOT_STATUS   = (1,0,4)
        self.SLOT_PROF     = (1,0,5)
        

    bindto = ['user.org:user:MagicSeqTop:1.0']

    def genAddr(self, bankId, rowIdx, colIdx):
        return (bankId << self.BL_BNK_ST) | (rowIdx << self.BL_ROW_ST) | (colIdx << self.BL_COL_ST)
    
    def genAddrForSlot(self, slotT, slotIdx):
        return self.genAddr(slotT[0], slotIdx, slotT[2])

    ###############################################
    ####### getter ################################
    ###############################################

    def getStatus(self):
        return self.read(self.genAddr(*self.REG_ST))
    def getMainCnt(self):
        return self.read(self.genAddr(*self.REG_MAINCNT))
    def getEndCnt(self):
        return self.read(self.genAddr(*self.REG_ENDCNT))
    def getDmaAddr(self):
        return self.read(self.genAddr(*self.REG_DMA_ADDR))
    def getDfxAddr(self):
        return self.read(self.genAddr(*self.REG_DFX_ADDR))
    
    def getSlot(self, slotIdx):

        addr_srcAddr  = self.genAddrForSlot(self.SLOT_SRC_ADDR, slotIdx)             
        addr_srcSz    = self.genAddrForSlot(self.SLOT_SRC_SIZE, slotIdx)           
        addr_desAddr  = self.genAddrForSlot(self.SLOT_DES_ADDR, slotIdx)             
        addr_desSz    = self.genAddrForSlot(self.SLOT_DES_SIZE, slotIdx)           
        addr_status   = self.genAddrForSlot(self.SLOT_STATUS  , slotIdx)            
        addr_prof     = self.genAddrForSlot(self.SLOT_PROF    , slotIdx)

        data_srcAddr          = self.read(addr_srcAddr)
        data_srcSz          = self.read(addr_srcSz)
        data_desAddr          = self.read(addr_desAddr)
        data_desSz          = self.read(addr_desSz)
        data_status          = self.read(addr_status)
        data_prof          = self.read(addr_prof)

        return data_srcAddr, data_srcSz, data_desAddr, data_desSz, data_status, data_prof
        

    ###############################################
    ####### setter ################################
    ###############################################

    def getStatus(self):
        return self.read(self.genAddr(*self.REG_ST))
    def getMainCnt(self):
        return self.read(self.genAddr(*self.REG_MAINCNT))
    def getEndCnt(self):
        return self.read(self.genAddr(*self.REG_ENDCNT))
    def getDmaAddr(self):
        return self.read(self.genAddr(*self.REG_DMA_ADDR))
    def getDfxAddr(self):
        return self.read(self.genAddr(*self.REG_DFX_ADDR))
    
    